# Chemprop 2.x Pure‑Python Workflow on Colab 🚀

*GPU‑Accelerated Graph Neural‑Network Regression Without the CLI*

### Warm‑Up Questions

**WQ‑1.** What is the main advantage of using **graph neural networks** for molecular property prediction compared to traditional machine learning approaches that use molecular descriptors?

<span style="color:cyan"><strong>Free response:</strong> YOUR RESPONSE TEXT HERE </span>

**WQ‑2.** Why might **scaffold-balanced splitting** be important when training a machine learning model on chemical data? What problems could arise if you use random splitting instead?

<span style="color:cyan"><strong>Free response:</strong> YOUR RESPONSE TEXT HERE </span>


## Learning Objectives
1. Use **Google Colab GPU** resources to train a Chemprop ≥ 2.0 model entirely from **Python code (no CLI flags)**.
2. Make **test‑set predictions** and create a **parity plot** with `matplotlib` / `seaborn`.

## 0 — Runtime & GPU Check
Make sure your Colab session uses a GPU (**Runtime → Change runtime type → GPU**).

In [ ]:
import torch, os, platform, sys
print('PyTorch :', torch.__version__)
print('CUDA?   :', torch.cuda.is_available())
if torch.cuda.is_available():
    print('Device  :', torch.cuda.get_device_name(0))
else:
    print('⚠️ Training will be CPU‑only (slow).')

### Install Chemprop 2.x

In [ ]:
# !pip install --quiet 'chemprop>=2.0' 

## 1 — Load the Delaney Solubility Dataset

In [ ]:
import pandas as pd, requests, io, os, math, numpy as np
url = 'https://raw.githubusercontent.com/deepchem/deepchem/master/datasets/delaney-processed.csv'
df = pd.read_csv(io.StringIO(requests.get(url).text))
df = df.rename(columns={'measured log solubility in mols per litre': 'solubility'})[['smiles','solubility']]
data_path = 'delaney.csv'
df.to_csv(data_path, index=False)
df.head()

## 2 — Scaffold‑Balanced Train/Val/Test Split

In [ ]:
from rdkit import Chem
from chemprop.data import MoleculeDatapoint, MoleculeDataset
from chemprop.data.splitting import make_split_indices          # NEW ✅

# ---- Build MoleculeDataset ----
dataset = MoleculeDataset([
    MoleculeDatapoint(smiles=[s], targets=[v])
    for s, v in zip(df.smiles, df.solubility)
])

# ---- Generate scaffold-balanced indices ----
mols = [Chem.MolFromSmiles(s) for s in df.smiles]               # RDKit molecules
train_idx, val_idx, test_idx = make_split_indices(
    mols,
    split='scaffold_balanced',
    sizes=(0.8, 0.1, 0.1),
    seed=42
)

# ---- Slice the MoleculeDataset ----
train = [dataset[i] for i in train_idx]
val   = [dataset[i] for i in val_idx]
test  = [dataset[i] for i in test_idx]

print(len(train), len(val), len(test))

## 3 — Train Chemprop (Python API Only)

In [ ]:
from chemprop.args import TrainArgs
from chemprop.train import run_training

args = TrainArgs()
# --- essential paths ---
args.train_data_path = train_path
args.val_data_path = val_path
args.save_dir = 'chemprop_model_py'

# --- basic settings ---
args.dataset_type = 'regression'
args.target_columns = ['solubility']
args.epochs = 30
args.batch_size = 32
args.gpu = 0 if torch.cuda.is_available() else -1
args.metric = 'rmse'
# You can tweak more hyperparameters here, e.g. args.hidden_size, args.depth

run_training(args)

## 4 — Predict on Held‑out Test Set

In [ ]:
from chemprop.train import load_checkpoint, make_predictions
from chemprop.args import PredictArgs

# Find the best checkpoint saved during training
ckpt_dir = args.save_dir
checkpoint_path = os.path.join(ckpt_dir, 'model_0', 'model.pt')

pred_args = PredictArgs()
pred_args.test_path = test_path
pred_args.checkpoint_paths = [checkpoint_path]
preds = make_predictions(pred_args)

# Combine predictions with truth
test_df = pd.read_csv(test_path)
test_df['pred'] = preds
test_df.head()

## 5 — Visualise Parity Plot

### Auto‑Graded Checkpoints

**Checkpoint CP‑1 (2 pts)** — Implement `calculate_rmse(y_true, y_pred)` that returns the root mean square error between true and predicted values.


In [ ]:
### BEGIN SOLUTION
import numpy as np

def calculate_rmse(y_true, y_pred):
    """Calculate root mean square error."""
    return np.sqrt(np.mean((y_true - y_pred)**2))
### END SOLUTION


In [ ]:
# hidden tests
import numpy as np
y_true = np.array([1, 2, 3, 4, 5])
y_pred = np.array([1.1, 2.1, 2.9, 4.1, 4.9])
rmse = calculate_rmse(y_true, y_pred)
assert abs(rmse - 0.141) < 0.01  # approximately 0.141


**Checkpoint CP‑2 (3 pts)** — Implement `create_parity_plot_data(y_true, y_pred)` that returns a pandas DataFrame with columns 'true' and 'predicted' for creating parity plots.


In [ ]:
### BEGIN SOLUTION
import pandas as pd
import numpy as np

def create_parity_plot_data(y_true, y_pred):
    """Create DataFrame for parity plot."""
    return pd.DataFrame({
        'true': y_true,
        'predicted': y_pred
    })
### END SOLUTION


In [ ]:
# hidden tests
import pandas as pd
import numpy as np
y_true = np.array([1, 2, 3])
y_pred = np.array([1.1, 2.1, 2.9])
df = create_parity_plot_data(y_true, y_pred)
assert len(df) == 3
assert 'true' in df.columns
assert 'predicted' in df.columns
assert df['true'].iloc[0] == 1
assert df['predicted'].iloc[0] == 1.1


In [ ]:
import matplotlib.pyplot as plt, seaborn as sns
sns.set(style='ticks')

plt.figure(figsize=(5,5))
sns.scatterplot(x='solubility', y='pred', data=test_df)
lims = [test_df[['solubility','pred']].min().min(), test_df[['solubility','pred']].max().max()]
plt.plot(lims, lims, '--k')
plt.xlabel('True log S')
plt.ylabel('Predicted log S')
plt.title('Chemprop 2.x Parity Plot (Python API)')
plt.show()

rmse = math.sqrt(((test_df.solubility - test_df.pred)**2).mean())
print(f'RMSE : {rmse:.3f}')

### Critical‑Thinking Questions

**CTQ‑1.** What are the potential **limitations** of using machine learning models for molecular property prediction? When might experimental validation still be necessary despite having a good-performing model?

<span style="color:cyan"><strong>Free response:</strong> YOUR RESPONSE TEXT HERE </span>


## Your Turn 📝
1. Swap in **your own dataset** (same columns: `smiles`, target).  
2. Experiment with `args.hidden_size`, `args.depth`, `args.dropout`.  
3. Try `args.ensemble_size = 5` for an ensemble model.  
4. Create residual plots or error vs. molecular weight.